# Initial Mocap Exploration

The purposes of this notebook are twofold:

* Exploring 1+ CMU Mocap data files:
    * IO
    * Data Structure
    * Feature Extraction
* Visualising the skeleton using RidgidBodyDynamics / MeshCat + additional work

## AMC files seem easiest
However, they don't contain skeleton info => We need the .asf file too. It seems that many authors work with BVH files instead and in order to be more flexible in reusing other work and pre-processing, perhaps BVH will be the way forward, rendering the below AMC parser not especially useful.

In [29]:
function read_amc_(fname, T::Type)
    
    # lookups for skeleton position
    lens =[6 3 3 3 3 3 3 2 3 1 1 2 1 2 2 3 1 1 2 1 2 3 1 2 1 3 1 2 1];
    locations = [1 7 10 13 16 19 22 25 27 30 31 32 34 35 37 39 42 43 44 46 47 49 52 53 55 56 59 60 62];
    
    # lookup for text strings in AMC file format
    skel_lkp = Dict(
        "root"=>1,
        "lowerback"=>2,
        "upperback"=>3,
        "thorax"=>4, 
        "lowerneck"=>5, 
        "upperneck"=>6, 
        "head"=>7,
        "rclavicle"=>8, 
        "rhumerus"=>9,
        "rradius"=>10,
        "rwrist"=>11,
        "rhand"=>12,
        "rfingers"=>13,
        "rthumb"=>14,
        "lclavicle"=>15, 
        "lhumerus"=>16,
        "lradius"=>17,
        "lwrist"=>18,
        "lhand"=>19,
        "lfingers"=>20,
        "lthumb"=>21,
        "rfemur"=>22,
        "rtibia"=>23,
        "rfoot"=>24,
        "rtoes"=>25,
        "lfemur"=>26,
        "ltibia"=>27,
        "lfoot"=>28,
        "ltoes"=>29
    )
    
    
    frame_nums, frames = open(fname) do f
        # ignore header
        while !eof(f) && (readline(f) != ":DEGREES") end
        frame_nums = Int32[]
        frames = Vector[]
        
        # main loop
        while !eof(f)
            
            # Each frame begins with a line of one integer of frame num
            xf = readline(f)
            push!(frame_nums, parse(Int, xf))
            
            # Next 29 lines will be the angle labels (arbitrary order allowed)
            frame = Vector{T}(undef, 62);
            for i in 1:29
                elems = split(readline(f))
                angle_ix = skel_lkp[elems[1]]
                loc = locations[angle_ix];
                len = lens[angle_ix];
                frame[loc:(loc+len-1)] = parse.(T, elems[2:end]);
            end
            push!(frames, frame)
        end
        
        (frame_nums, frames)
    end
    
    frame_nums, reduce(hcat, frames)
end

read_amc(fname, T::Type) = begin; fnums, fs = read_amc_(fname, T); @assert all(diff(fnums) .== 1); fs; end
read_amc(fname) = read_amc(fname, Float64)

read_amc (generic function with 2 methods)

In [71]:
X = read_amc("../data/mocap/cmu/walk/02_01.amc")

62×343 Array{Float64,2}:
  10.4194     10.4117      10.4037    …   11.0359     11.0323      11.0237  
  16.7048     16.684       16.6749        17.5068     17.4953      17.502   
 -30.1003    -29.9168     -29.7319        29.051      29.2606      29.4538  
  -2.48972    -2.69729     -2.73684       -4.01745    -4.41668     -4.48244 
  -9.82194   -10.0082     -10.1104         6.83579     7.13935      6.81263 
  -3.00914    -2.83243     -2.67905   …    0.112677   -0.642809    -1.59521 
   7.86573     7.44596      7.31519        5.78481     6.08656      5.7506  
   1.57272     1.65374      1.74924       -1.59455    -1.74843     -1.88034 
   6.3571      6.09331      5.93159       -0.699558    0.1647       0.989992
   3.47255     3.91254      3.96845        1.79665     1.59999      1.78225 
   2.55226     2.58553      2.68906   …   -2.12702    -2.27025     -2.40255 
   1.65684     1.67171      1.61541        1.9459      2.15737      2.42913 
  -0.599593    0.0784557    0.211031      -1.29197 

In [59]:
using Formatting, DelimitedFiles

### Get list of just locomotion types

In [67]:
# cmu_locomotion_lkp is simply a dump of the text (rendered html) of the CMU locomotion style sections
# (there are four of these: running, jumping, walking, varied).
error("Already run, and source file no longer exists")
cmu_loco_f, cmu_loco_type = open("../data/mocap/cmu_locomotion_lkp") do f
    
    ids = String[]
    descr = String[]
    
    # main loop
    while !eof(f)

        # Each frame begins with a line of one integer of frame num
        xhead = split(readline(f))
        @assert xhead[1] == "Trial"
        subject = parse(Int, xhead[4])
        
        while true
            x = readline(f)
            isempty(strip(x)) && break
            elems = split(x)
            push!(ids, format("{:02d}_{:02d}", subject, elems[1]))
            push!(descr, elems[2])
        end
    end

    ids, descr
end

open("../data/mocap/cmu_locomotion_lkp.txt", "w") do io
    reorder = sortperm(cmu_loco_f)
    writedlm(io, [cmu_loco_f[reorder] cmu_loco_type[reorder]])
end

ErrorException: Already run, and source file no longer exists